In [1]:
import pandas as pd
import numpy as np

np.float_ = np.float64

from prophet import Prophet
import requests
from datetime import datetime, timedelta
import duckdb as db

m = Prophet()
conn = db.connect("file.db")
conn.sql("""
    CREATE SEQUENCE seq_id START 1;
    CREATE TABLE contract_call (y INTEGER, ds DATETIME UNIQUE)
""")

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Get data

end_time = datetime.now()
print(end_time.timestamp())
start_time = (end_time - timedelta(seconds=3600.0)).timestamp()
print(start_time)

transaction_types = ["CONTRACTCALL"]

from typing import Union
from pprint import pprint

def get_data(start_time: float | int, limit: int, transaction_type: str, order: Union["asc", "desc"]):
    url = f"https://testnet.mirrornode.hedera.com/api/v1/transactions?limit={limit}&order={order}&timestamp=gte%3A{start_time}&transactiontype={transaction_type}"
    data = requests.get(url).json()
    transactions = data["transactions"]
    new_transactions: list[dict | None] = []
    for transaction in transactions:
        new_data = {}
        if not transaction["scheduled"] and transaction["result"] == "SUCCESS":
            _, seconds, nanoseconds = transaction["transaction_id"].split("-")
            # new_data["id"] = transaction["transaction_id"]
            new_data["charged_tx_fee"] = transaction["charged_tx_fee"]
            new_data["time_tx_occured"] = datetime.fromtimestamp(float(f"{seconds}.{nanoseconds}")).isoformat()

            new_transactions.append(new_data)
    return new_transactions


transactions: list[dict | None] = get_data(start_time, 100, transaction_types[0], "asc")
pprint(transactions)

1723991326.134706
1723987726.134706
[{'charged_tx_fee': 12720000, 'time_tx_occured': '2024-08-18T13:32:35.928693'},
 {'charged_tx_fee': 12720000, 'time_tx_occured': '2024-08-18T13:34:41.512255'},
 {'charged_tx_fee': 127200000, 'time_tx_occured': '2024-08-18T13:34:48.718056'}]


In [3]:
try:
    for transaction in transactions:
        conn.sql(f"""
            INSERT INTO contract_call VALUES ('{transaction["charged_tx_fee"]}', '{transaction["time_tx_occured"]}')
        """)
except db.ConstraintException:
    pass

In [4]:
conn.sql("""
    SELECT * FROM contract_call
""").show()

┌───────────┬────────────────────────────┐
│     y     │             ds             │
│   int32   │         timestamp          │
├───────────┼────────────────────────────┤
│  12720000 │ 2024-08-18 13:32:35.928693 │
│  12720000 │ 2024-08-18 13:34:41.512255 │
│ 127200000 │ 2024-08-18 13:34:48.718056 │
└───────────┴────────────────────────────┘



In [5]:
df = conn.sql("""
    SELECT * FROM contract_call
""").to_df()
df.head()

,y,ds
0,12720000,2024-08-18 13:32:35.928693
1,12720000,2024-08-18 13:34:41.512255
2,127200000,2024-08-18 13:34:48.718056


In [6]:
m.fit(df)
future = m.make_future_dataframe(periods=1, freq="H")

14:28:46 - cmdstanpy - INFO - Chain [1] start processing
14:28:46 - cmdstanpy - INFO - Chain [1] done processing
/home/codespace/.python/current/lib/python3.10/site-packages/prophet/forecaster.py:1854: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(


In [7]:
forecast = m.predict(future)
forecast.tail()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2024-08-18 13:32:35.928693,9.872475e+06,-4.640350e+07,6.139757e+07,9.872475e+06,9.872475e+06,0.0,0.0,0.0,0.0,0.0,0.0,9.872475e+06
1,2024-08-18 13:34:41.512255,6.964638e+07,1.672447e+07,1.218496e+08,6.964638e+07,6.964638e+07,0.0,0.0,0.0,0.0,0.0,0.0,6.964638e+07
2,2024-08-18 13:34:48.718056,7.307612e+07,2.321815e+07,1.243597e+08,7.307612e+07,7.307612e+07,0.0,0.0,0.0,0.0,0.0,0.0,7.307612e+07
3,2024-08-18 14:34:48.718056,1.786565e+09,1.733023e+09,1.840816e+09,1.786565e+09,1.786565e+09,0.0,0.0,0.0,0.0,0.0,0.0,1.786565e+09
